In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import imdb

- IMDB 리뷰 데이터는 기존 데이터 셋과는 달리 이미 훈련 데이터와 테스트 데이터를 50:50 비율로 구분해서 제공
- imdb.data_load()의 인자로 num_words를 사용하면 이 데이터에서 등장 빈도 순위로 몇 등까지의 단어를 사용할 것인지를 의미한다.
- 예를들어 10,000을 넣으면, 등장 빈도 순위가 1~10,000에 해당하는 단어만 사용한다.

In [ ]:
(X_train, y_train), (X_test, y_test) = imdb.load_data()

print('훈련용 리뷰 개수: {}'.format(len(X_train)))
print('테스트용 리뷰 개수: {}'.format(len(X_test)))
num_classes = len(set(y_train))
print('카테고리: {}'.format(num_classes))

17473536/17464789 [==============================] - 0s 0us/step
훈련용 리뷰 개수: 25000
테스트용 리뷰 개수: 25000
카테고리: 2


In [ ]:
print('첫번째 훈련용 리뷰 :', X_train[0])
print('첫번째 훈련용 리뷰의 레이블 :', y_train[0])

첫번째 훈련용 리뷰 : [1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 22665, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 21631, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 31050, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]
첫번째 훈련용 리뷰의 레이블 : 1


- 케라스의 Embedding()은 단어 각각에 대해 정수로 변환된 입력에 대해서 임베딩 작업을 수행한다.

- 단어 각각에 정수를 부여하는 방법으로는 단어를 빈도수 순대로 정렬하고 순차적으로 정수를 부여하는 방법이 있다. 로이터 뉴스와 IMDB 리뷰 데이터는 방법을 사용하였으며 이미 이 작업이 끝난 상태이다.

- 등장 빈도 순으로 단어를 정렬하여 정수를 부여하였을 때의 장점은 등장 빈도수가 적은 단어의 제거이다. 예를 들어서 25,000개의 단어가 있다고 가정하고, 해당 단어를 등장 빈도수 순가 높은 순서로 1부터 25,000까지 정수를 부여했다고 하자. 이렇게 되면 등장 빈도 순으로 등수가 부여된 것과 다름없으므로 전처리 작업에서 1,000보다 큰 정수로 맵핑된 단어들을 제거한다면 등장 빈도 상위 1,000개의 단어만 남길 수 있다.

In [ ]:
# 단어 집합의 크기를 10,000으로 제한하고, 리뷰 최대 길이는 500으로 제한하여 패딩을 진행
import re
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU, Embedding
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model

vocab_size = 10000
max_len = 500

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=vocab_size)

X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [ ]:
embedding_dim = 100
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(GRU(hidden_units))
model.add(Dense(1, activation='sigmoid'))

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint('GRU_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
history = model.fit(X_train, y_train, epochs=15, callbacks=[es, mc], batch_size=64, validation_split=0.2)

Epoch 1/15
312/313 [============================>.] - ETA: 0s - loss: 0.5099 - acc: 0.7586
Epoch 1: val_acc improved from -inf to 0.76940, saving model to GRU_model.h5
313/313 [==============================] - 16s 31ms/step - loss: 0.5098 - acc: 0.7585 - val_loss: 0.4884 - val_acc: 0.7694
Epoch 2/15
312/313 [============================>.] - ETA: 0s - loss: 0.3308 - acc: 0.8698
Epoch 2: val_acc improved from 0.76940 to 0.87560, saving model to GRU_model.h5
313/313 [==============================] - 9s 29ms/step - loss: 0.3306 - acc: 0.8698 - val_loss: 0.3097 - val_acc: 0.8756
Epoch 3/15
311/313 [============================>.] - ETA: 0s - loss: 0.2543 - acc: 0.9031
Epoch 3: val_acc improved from 0.87560 to 0.88160, saving model to GRU_model.h5
313/313 [==============================] - 9s 29ms/step - loss: 0.2543 - acc: 0.9032 - val_loss: 0.3290 - val_acc: 0.8816
Epoch 4/15
311/313 [============================>.] - ETA: 0s - loss: 0.2096 - acc: 0.9218
Epoch 4: val_acc improved from 0

In [ ]:
loaded_model = load_model('GRU_model.h5')
print("\n 테스트 정확도: %.4f" % (loaded_model.evaluate(X_test, y_test)[1]))

782/782 [==============================] - 8s 9ms/step - loss: 0.3145 - acc: 0.8847

 테스트 정확도: 0.8847
